In [3]:
import pandas as pd
import numpy as np
from decimal import Decimal
from llms import gemini
from llms import chatGPT
from sklearn.metrics.pairwise import cosine_similarity
import json
import time
# from lemmatize import lemmatize
x_chat = chatGPT()
x_gemini = gemini()




In [4]:
a = "Beyond Limits is an AI technology company focused on developing Cognitive AI solutions for demanding industries like energy, utilities, healthcare, and finance. Their technology combines human knowledge with machine learning to provide decision makers with insights, guidance, and recommendations for optimizing operations and increasing profitability."
b = 'Financial managers that prepare operational or risk reports for management analysis.'
a = x_chat.get_embedding(a)
b = x_chat.get_embedding(b)
cosine_similarity([a],[b])

array([[0.14830178]])

In [56]:
df2[["generated_description","Tasks/Jobs","Customers","Industry"]].iloc[0].values

array(['Pika.art is an AI-powered platform that allows users to create and edit films using just captions and still images. By leveraging advanced AI algorithms, Pika simplifies the video creation process, making it accessible to a wider audience.',
       'Video editing, Filmmaking, Storyboarding, Content creation',
       'Filmmakers, Content creators, Marketers', 'Video production'],
      dtype=object)

In [50]:
df[["Tasks/Jobs","Customers","Industry"]].head()

,Tasks/Jobs,Customers,Industry
0,"mRNA drug discovery, High-throughput screening...","Pharmaceutical companies, Research institutions",Pharmaceutical
1,"UX/UI Design, Web Development, Mobile App Deve...","Startups, Businesses, Enterprises",Digital Product Development
2,"Video ad creation, Campaign optimization, Perf...","Streaming platforms, Television networks, Brands",Digital advertising
3,"Peer support, relapse prevention, access to re...",Individuals in recovery from substance use dis...,Addiction Recovery
4,"Visual product search, 3D product visualizatio...","Fashion brands, Online retailers",Apparel e-commerce


In [52]:
df = pd.read_csv('../output/df_with_examples.csv')
df = df.dropna(subset=["Example1"])

In [37]:

job_cols = [[f"Example_Description{i}",f"Example_Job_Title{i}"] for i in range(1,4)]
for job in job_cols[0:]:
    for col in job:
        df = x_chat.run_batch_embeddings(df,col)

df.to_csv('../output/df_with_examples_embeddings.csv', index=False)

Processing final batch 0 to 10
Processing final batch 0 to 10
Processing final batch 0 to 10
Processing final batch 0 to 10
Processing final batch 0 to 10
Processing final batch 0 to 10


In [ ]:
# random_descriptions = df_exp['description'].sample(10, random_state=2)
# print("\n".join(random_descriptions.values))


In [5]:
df_exp = pd.read_csv('../input/gpts_labels/gpt_exposure_embeddings.csv')

df_exp.task_embedding = df_exp.task_embedding.apply(lambda x: x.strip("[]").split(", "))

task_embeddings = []
for x in df_exp.task_embedding:
    task_embeddings.append([float(y) for y in x])
df_exp.task_embedding = task_embeddings


task_embeddings = np.array(task_embeddings)
task_embeddings = np.vstack(task_embeddings)


df_exp.title_embedding = df_exp.title_embedding.apply(lambda x: x.strip("[]").split(", "))
title_embeddings = []
for x in df_exp.title_embedding:
    title_embeddings.append([float(y) for y in x])
df_exp.title_embedding = title_embeddings


task_embeddings.shape

(11582, 3072)

In [6]:
df = pd.read_csv('../output/df_with_examples_embeddings.csv')


for i in range(3):
    df[f"Example_Description{i+1}_embedding"] = df[f"Example_Description{i+1}_embedding"].apply(lambda x: [float(y) for y in x.strip("[]").split(", ")])
    df[f"Example_Job_Title{i+1}_embedding"] = df[f"Example_Job_Title{i+1}_embedding"].apply(lambda x: [float(y) for y in x.strip("[]").split(", ")])




In [38]:

results = []
# results = list(pd.read_csv("../output/onet_df.csv").values)

for index, row in list(df.iterrows())[0:]:
    print("***********************")
    print(f"Processing {index}")
    name = row["organization name"]
    print(name)
    print(row["generated_description"])
        
    job_results = []
    for i in range(3): 
        example = row[f"Example{i+1}"]
        description = row[f'Example_Description{i+1}']
        job_title = row[f'Example_Job_Title{i+1}']
        print(f"\nProcessing Example: {example} \nJob: {description}\n")

        job_embedding = np.array(row[f'Example_Description{i+1}_embedding']).reshape(1,-1)
        job_title_embedding = np.array(row[f'Example_Job_Title{i+1}_embedding']).reshape(1,-1)
        cosine_sim = cosine_similarity(job_embedding, task_embeddings)
        
        top3 = sorted(zip(cosine_sim[0], df_exp[['title', 'Task', 'description','title_embedding']].values), reverse=True)[:3]
        for cosine_sim_job, onet in top3:
            print(cosine_sim_job)
            onet_title_embedding = np.array([onet[3]])
            cosine_sim_title =cosine_similarity(job_title_embedding, onet_title_embedding)[0][0]
            print(f"{onet[2]} \nCosine Similarity: {cosine_sim_job}, Person Cosine Similarity: {cosine_sim_title}")
            results.append([name,example,row[f"situation{i+1}_conf_interval"], row[f"situation{i+1}_conf_interval_reasoning"],job_title,onet[0],onet[1],job_title, cosine_sim_job,cosine_sim_title])
 

    if index % 50 == 0:
        print("**Saving Results**\n\n")
        onet_df = pd.DataFrame(results, columns=["organization_name","example","situation_conf_interval", "situation_conf_interval_reasoning","job","onet_title","onet_task","example_job_title","task_similarity", "job_title_similarity"])
        onet_df.situation_conf_interval = onet_df.situation_conf_interval.astype("float")
        onet_df.to_csv("../output/onet_df.csv", index=False)
        time.sleep(10)
        
    print("***********************\n\n")


onet_df = pd.DataFrame(results, columns=["organization_name","example","situation_conf_interval", "situation_conf_interval_reasoning","job","onet_title","onet_task","example_job_title","task_similarity", "job_title_similarity"])
onet_df.situation_conf_interval = onet_df.situation_conf_interval.astype("float")
onet_df.to_csv("../output/onet_df.csv", index=False)

***********************
Processing 0
Anima Biotech
Anima Biotech develops novel, selective small molecule mRNA drugs using a high-throughput automated phenotypic screening platform coupled with AI-driven mechanism of action elucidation. They are advancing a broad pipeline of drug candidates across 18 therapeutic areas through strategic partnerships with pharma companies like Lilly and Takeda.

Processing Example: A research scientist at a pharmaceutical company uses Anima Biotech's platform to screen thousands of small molecules for their ability to modulate a specific mRNA target associated with a rare disease. This helps identify potential drug candidates with high selectivity and efficacy. 
Job: Research Scientist that conduct experiments to test the efficacy and safety of potential drug candidates.

0.6695373973071153
Biochemists and biophysicists that develop or test new drugs or medications intended for commercial distribution. 
Cosine Similarity: 0.6695373973071153, Person Cosin

In [55]:
onet_df.iloc[0]

organization_name                                                          Luming Labs
example                              A research scientist at a pharmaceutical compa...
situation_conf_interval                                                            9.0
situation_conf_interval_reasoning    The scientist needs to analyze large datasets ...
job                                         [Example_Description3, Example_Job_Title3]
onet_title                                                   Natural Sciences Managers
onet_task                            Plan or direct research, development, or produ...
example_job_title                                                  Research Scientists
task_similarity                                                               0.656189
job_title_similarity                                                          0.583292
Name: 0, dtype: object

In [9]:
curr = pd.read_csv("../output/onet_df.csv")

print(curr.iloc[-1])
df[df["organization name"]=="Obviously AI"]



organization_name                                                         Obviously AI
example                              A software developer wants to create a chatbot...
situation_conf_interval                                                            7.0
situation_conf_interval_reasoning    This example is less confident as the task is ...
job                                     ['Example_Description3', 'Example_Job_Title3']
onet_title                                                         Software Developers
onet_task                            Confer with systems analysts, engineers, progr...
example_job_title                                                  Software Developers
task_similarity                                                               0.707505
job_title_similarity                                                          0.999999
Name: 8099, dtype: object


,organization name,num employees,founded date,full_description,industries,headquarters location,description,cb rank,postal code,headquarters region,...,Example_Job_Title3,Example_Description3,situation3_conf_interval,situation3_conf_interval_reasoning,Example_Description1_embedding,Example_Job_Title1_embedding,Example_Description2_embedding,Example_Job_Title2_embedding,Example_Description3_embedding,Example_Job_Title3_embedding
899,Obviously AI,1-10,2020-02-02,"At Obviously AI, our vision is to turn every c...","Artificial Intelligence (AI), Machine Learning...","San Francisco, California, United States",Obviously AI enables businesses to build custo...,"3,959",94105,www.obviously.ai,...,Software Developers,"Software Developers that design, develop, or t...",7.0,This example is less confident as the task is ...,"[-0.016252316534519196, -0.027554688975214958,...","[-0.005456793587654829, -0.015291607938706875,...","[-0.03545496240258217, -0.025911446660757065, ...","[-0.018091563135385513, -0.01092542428523302, ...","[-0.008595336228609085, 0.039918918162584305, ...","[-0.002831327961757779, 0.02374046854674816, -..."


In [10]:
s1 = "Animators and Multimedia Artists that create storyboards to illustrate scenes or action sequences for films, television programs, or other media."
s2 = "Special effects artists and animators that apply story development, directing, cinematography, and editing to animation to create storyboards that show the flow of the animation and map out key scenes and characters."



In [13]:
e1 = x_chat.get_embedding(s1)
#e1 = df_exp[df_exp.description.str.contains("managers that evaluate the organization")].task_embedding.iloc[0]
e2 = x_chat.get_embedding(s2)

#e2 = x_chat.get_embedding("Computer Information Systems Managers evaluate cloud service offering different provider select suitable option company 's need")
e1 = np.array(e1).reshape(1,-1)
e2 = np.array(e2).reshape(1,-1)
cosine_similarity(e1,e2, 'cosine')[0][0]

0.6625274673336069

In [17]:
title_embeddings = np.array(df_exp.title_embedding)
title_embeddings = np.vstack(title_embeddings)

In [147]:
onet_df = pd.read_csv("../output/results v3/onet_df.csv")
onet_df = onet_df[[col for col in onet_df.columns if "titles" not in col]]
onet_df[["Job_similarity_titles"]]= 'N/A'
len(onet_df)

6093